Descargamos las librerias

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

Paso 1: Cargamos el archivo html y creamos una cabecera para que la página no crea que somos bots

In [2]:
# Agregar encabezados de usuario para emular una solicitud desde un navegador
headers = {
    "User-Agent":"My Phyton App"
}

with open(r"C:\Users\Geovanny\Desktop\septimo_semestre\Recuperacion_Informacion_2024A\Deberes_RI\link_recetas\Recipes A-Z _ Allrecipes.com.html","r",encoding="utf-8") as file: 
    html_content1=file.read()

soup= BeautifulSoup(html_content1,"html.parser" )
#soup

Funcion para guardar los archivos html de cada link que se pase

In [3]:
def responseURL(urlresponse,headers,timeout=10):
    html_content=[]
    for url1 in tqdm(urlresponse, "Progreso"):
        try:
            response = requests.get(url1, headers=headers, timeout=timeout)
            htmltext = response.text
            html_content.append(BeautifulSoup(htmltext, "html.parser"))
        except requests.Timeout:
            print(f"Timeout occurred for URL: {url1}")
        except requests.RequestException as e:
            print(f"Request failed for URL: {url1} with exception: {e}")
    return html_content    

Funcion para obtener los links del html Recipes A-Z_Allrecipes.com.html

Obtenemos los links que nos dirigen a cada pagina que tiene recetas de un tipo especifico, Por ejemplo la pagina de recetas de pescado tendra varios links para cada tipo de forma de prepararlo.

In [4]:
def obtenerLinks(htmlLinks):
    totalLinks=[]
    for itemLink in htmlLinks.find_all("li", class_="comp mntl-link-list__item"):
        a_tag = itemLink.find("a", href=True)
        if a_tag:
            totalLinks.append(a_tag['href'])
    return totalLinks    


Obtenemos los links para todos los diferentes tipos de recetas que hay.

In [5]:
def recetasPorLink(HtmlPag):
    totalLinkspag=[]
    for itemLink in HtmlPag.find_all("a", class_="comp mntl-card-list-items mntl-document-card mntl-card card card--no-image"):
        if itemLink.has_attr('href'):
            totalLinkspag.append(itemLink['href'])
    return totalLinkspag     
    

Función para obtener el titulos de cada receta

In [6]:
def obtenerTitulo(HtmlTitulo):
    titulorespuesta=[]
    for tituloreceta in HtmlTitulo:
        titulorespuesta.append(tituloreceta.find("title").text)
    return titulorespuesta

Función para obtener la descripción de cada receta

In [7]:
def descripcionReceta(HTMLReceta):
    respuestaDescripcion=[]
    for descripcionReceta in HTMLReceta:
        resp =descripcionReceta.find("p", class_="article-subheading type--dog")
        if resp:
           respuestaDescripcion.append(resp.text)
        else:
           respuestaDescripcion.append(np.nan)
    return respuestaDescripcion

Función para obtener los ingredientes de cada receta

In [8]:
def ingredienteReceta(HTMLIngrediente):
    ingrediente=[]
    for ingredienteItem in HTMLIngrediente:
        respuestaIngrediente=''
        for item in ingredienteItem.find_all("li", class_="mm-recipes-structured-ingredients__list-item"):
             respuestaIngrediente += item.text.strip()+'\n'
        ingrediente.append(respuestaIngrediente)
    return ingrediente

Función para obtener los pasos de cada receta

In [9]:
def pasosReceta(HTMLPasos):
    pasos=[]
    for pasosItem in HTMLPasos:
        respuestaPasos=''
        for itemPaso in pasosItem.find_all("li", class_="comp mntl-sc-block mntl-sc-block-startgroup mntl-sc-block-group--LI"):
            for tag in itemPaso.find_all(["figure", "div"]):
                tag.decompose()
            respuestaPasos+=itemPaso.text.strip()+'\n'
        pasos.append(respuestaPasos)
    return pasos

Obtenemos los links de cada tipo de receta que hay del html Recipes A-Z_Allrecipes.com.html, y luego obtenemos los html de cada tipo de receta

In [10]:
linksHtml= obtenerLinks(soup)
#linksHtml

In [11]:
HTMLrecetas=responseURL(linksHtml,headers)
#linksrecetas

Progreso: 100%|██████████| 378/378 [03:46<00:00,  1.67it/s]


De cada tipo de receta obtenemos los links y los guardamos en una lista.

In [12]:
linksPorPaginaReceta= [recetasPorLink(link) for link in HTMLrecetas]
#linksPorPaginaReceta

In [13]:
linksTodasPaginas= [] 
for desp in linksPorPaginaReceta:
    for des in desp:
        linksTodasPaginas.append(des)

#linksTodasPaginas

Cantidad de links de todas las recetas

In [14]:
len(linksTodasPaginas)

18124

In [14]:
linksTodasPaginas[4312]

'https://www.allrecipes.com/recipe/12291/cherry-crumble/'

Obtenemos los html de cada link de todos los tipos de recetas que hay

In [15]:
#linksrecetas=responseURL(linksTodasPaginas[:367],headers)
HTMLrecetasTodas=responseURL(linksTodasPaginas,headers)
#linksrecetas

Progreso:  56%|█████▌    | 10063/18124 [1:36:27<7:29:03,  3.34s/it]  

Timeout occurred for URL: https://www.allrecipes.com/teriyaki-steak-quesadillas-recipe-7972550


Progreso: 100%|██████████| 18124/18124 [5:01:23<00:00,  1.00it/s]      


Llamamos a la función obtener Titulo para obtener el titulo de cada receta y lo guardamos en un dataframe en la columna 'Titulo'

In [18]:
titulo_receta = obtenerTitulo(HTMLrecetasTodas)
#titulo_receta

In [19]:
cocina=pd.DataFrame(titulo_receta, columns=['Titulo'])
cocina

,Titulo
0,Air Fryer Smashed Potatoes Recipe
1,Air Fryer Quesadillas Recipe
2,Air Fryer Truffle Polenta Fries Recipe
3,Air Fryer Firecracker Salmon Bites Recipe
4,Air Fryer Chicken Bites Recipe
...,...
18118,Vegan Zucchini Banana Bread Recipe
18119,Zucchini-Raspberry Bread Recipe
18120,Healthier Mom's Zucchini Bread Recipe
18121,"Zucchini Bread, Pumpkin Style Recipe"


Llamamos a la función descripcionReceta para obtener la descripcion de cada receta y lo guardamos en un dataframe en la columna 'descripcion'

In [20]:
descripcion=descripcionReceta(HTMLrecetasTodas)

In [21]:
cocina['descripcion']=descripcion
cocina

,Titulo,descripcion
0,Air Fryer Smashed Potatoes Recipe,"These golden, crispy air fryer smashed potatoe..."
1,Air Fryer Quesadillas Recipe,These air fryer quesadillas are golden and cri...
2,Air Fryer Truffle Polenta Fries Recipe,"These air fryer truffle polenta fries, flavore..."
3,Air Fryer Firecracker Salmon Bites Recipe,These air fryer firecracker salmon bites get a...
4,Air Fryer Chicken Bites Recipe,"These air fryer chicken bites, seasoned with l..."
...,...,...
18118,Vegan Zucchini Banana Bread Recipe,"This yummy, moist, rich zucchini banana bread ..."
18119,Zucchini-Raspberry Bread Recipe,It's a simple zucchini nut bread with a splash...
18120,Healthier Mom's Zucchini Bread Recipe,We packed even more zucchini into Mom's wonder...
18121,"Zucchini Bread, Pumpkin Style Recipe","Although I love zucchini bread, I wanted a new..."


Llamamos a la función ingredienteReceta para obtener los ingredientes de cada receta y lo guardamos en un dataframe en la columna 'ingrediente'

In [22]:
ingredientes = ingredienteReceta(HTMLrecetasTodas)

In [23]:
cocina['ingredientes']=ingredientes
cocina

,Titulo,descripcion,ingredientes
0,Air Fryer Smashed Potatoes Recipe,"These golden, crispy air fryer smashed potatoe...",8 ounces baby gold potatoes\n1 tablespoon melt...
1,Air Fryer Quesadillas Recipe,These air fryer quesadillas are golden and cri...,2 flour tortillas\n1/2 cup shredded cheese\nno...
2,Air Fryer Truffle Polenta Fries Recipe,"These air fryer truffle polenta fries, flavore...",1 (18 ounce) tube prepared polenta\n1 1/2 tabl...
3,Air Fryer Firecracker Salmon Bites Recipe,These air fryer firecracker salmon bites get a...,1/4 cup balsamic vinegar\n1/4 cup brown sugar\...
4,Air Fryer Chicken Bites Recipe,"These air fryer chicken bites, seasoned with l...","1 pound skinless, boneless chicken breasts, cu..."
...,...,...,...
18118,Vegan Zucchini Banana Bread Recipe,"This yummy, moist, rich zucchini banana bread ...",3 cups all-purpose flour\n1 teaspoon salt\n1 t...
18119,Zucchini-Raspberry Bread Recipe,It's a simple zucchini nut bread with a splash...,1 ½ cups self-rising flour\n1 teaspoon ground ...
18120,Healthier Mom's Zucchini Bread Recipe,We packed even more zucchini into Mom's wonder...,1 ½ cups all-purpose flour\n1 ½ cups white who...
18121,"Zucchini Bread, Pumpkin Style Recipe","Although I love zucchini bread, I wanted a new...","3 medium zucchini, cut into chunks\n4 ¾ cups a..."


Llamamos a la función pasosReceta, para obtener los pasos de cada receta y lo guardamos en un dataframe en la columna 'Pasos'

In [24]:
pasos = pasosReceta(HTMLrecetasTodas)

In [25]:
cocina['Pasos']=pasos
cocina

,Titulo,descripcion,ingredientes,Pasos
0,Air Fryer Smashed Potatoes Recipe,"These golden, crispy air fryer smashed potatoe...",8 ounces baby gold potatoes\n1 tablespoon melt...,Preheat an air fryer to 400 degrees F (200 deg...
1,Air Fryer Quesadillas Recipe,These air fryer quesadillas are golden and cri...,2 flour tortillas\n1/2 cup shredded cheese\nno...,"Heat tortillas in the microwave until pliable,..."
2,Air Fryer Truffle Polenta Fries Recipe,"These air fryer truffle polenta fries, flavore...",1 (18 ounce) tube prepared polenta\n1 1/2 tabl...,Preheat an air fryer to 400 degrees F (200 deg...
3,Air Fryer Firecracker Salmon Bites Recipe,These air fryer firecracker salmon bites get a...,1/4 cup balsamic vinegar\n1/4 cup brown sugar\...,"Combine balsamic vinegar, brown sugar, oil, so..."
4,Air Fryer Chicken Bites Recipe,"These air fryer chicken bites, seasoned with l...","1 pound skinless, boneless chicken breasts, cu...",Preheat an air fryer to 350 degrees F (175 deg...
...,...,...,...,...
18118,Vegan Zucchini Banana Bread Recipe,"This yummy, moist, rich zucchini banana bread ...",3 cups all-purpose flour\n1 teaspoon salt\n1 t...,Preheat the oven to 325 degrees F (165 degrees...
18119,Zucchini-Raspberry Bread Recipe,It's a simple zucchini nut bread with a splash...,1 ½ cups self-rising flour\n1 teaspoon ground ...,Preheat an oven to 350 degrees F (175 degrees ...
18120,Healthier Mom's Zucchini Bread Recipe,We packed even more zucchini into Mom's wonder...,1 ½ cups all-purpose flour\n1 ½ cups white who...,Preheat oven to 325 degrees F (165 degrees C)....
18121,"Zucchini Bread, Pumpkin Style Recipe","Although I love zucchini bread, I wanted a new...","3 medium zucchini, cut into chunks\n4 ¾ cups a...",Preheat an oven to 350 degrees F (175 degrees ...


Eliminamos las filas que tengan valores nulos

In [31]:
cocina = cocina.dropna()
cocina

,Titulo,descripcion,ingredientes,Pasos
0,Air Fryer Smashed Potatoes Recipe,"These golden, crispy air fryer smashed potatoe...",8 ounces baby gold potatoes\n1 tablespoon melt...,Preheat an air fryer to 400 degrees F (200 deg...
1,Air Fryer Quesadillas Recipe,These air fryer quesadillas are golden and cri...,2 flour tortillas\n1/2 cup shredded cheese\nno...,"Heat tortillas in the microwave until pliable,..."
2,Air Fryer Truffle Polenta Fries Recipe,"These air fryer truffle polenta fries, flavore...",1 (18 ounce) tube prepared polenta\n1 1/2 tabl...,Preheat an air fryer to 400 degrees F (200 deg...
3,Air Fryer Firecracker Salmon Bites Recipe,These air fryer firecracker salmon bites get a...,1/4 cup balsamic vinegar\n1/4 cup brown sugar\...,"Combine balsamic vinegar, brown sugar, oil, so..."
4,Air Fryer Chicken Bites Recipe,"These air fryer chicken bites, seasoned with l...","1 pound skinless, boneless chicken breasts, cu...",Preheat an air fryer to 350 degrees F (175 deg...
...,...,...,...,...
18118,Vegan Zucchini Banana Bread Recipe,"This yummy, moist, rich zucchini banana bread ...",3 cups all-purpose flour\n1 teaspoon salt\n1 t...,Preheat the oven to 325 degrees F (165 degrees...
18119,Zucchini-Raspberry Bread Recipe,It's a simple zucchini nut bread with a splash...,1 ½ cups self-rising flour\n1 teaspoon ground ...,Preheat an oven to 350 degrees F (175 degrees ...
18120,Healthier Mom's Zucchini Bread Recipe,We packed even more zucchini into Mom's wonder...,1 ½ cups all-purpose flour\n1 ½ cups white who...,Preheat oven to 325 degrees F (165 degrees C)....
18121,"Zucchini Bread, Pumpkin Style Recipe","Although I love zucchini bread, I wanted a new...","3 medium zucchini, cut into chunks\n4 ¾ cups a...",Preheat an oven to 350 degrees F (175 degrees ...


Guardamos en un CSV y un xlsx, el dataframe obtenido

In [32]:
# Guardar el DataFrame en un archivo CSV
cocina.to_csv('Deber12_Recetas.csv', index=True)

In [33]:
cocina.to_excel('Deber12_Recetas.xlsx', index=False)